<a href="https://colab.research.google.com/github/hmh100298/keras_tuner/blob/main/keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential

In [73]:
!pip install keras-tuner --upgrade

In [74]:
import keras_tuner as kt
from keras_tuner.tuners import Hyperband

(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()  ### du lieu quan ao tu fashion mnist
# chuan hoa gia tri pixel ve [0,1]
img_train = img_train.astype('float32')/255.0
img_test = img_test.astype('float32')/255.0

In [80]:
# hyper_model: build a model for hypertuning, hypertuning: quá trình lựa chọn quyền thiết lập các siêu tham số cho việc học máy tính của bạn ứng dụng (ML) 
#The model you set up for hypertuning is called a hypermodel.
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28,28))) #flatten : lam phang
# Tune the number of units in the first Dense layer
# Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10)) # 10 neural
# Tune the learning rate for the optimizer
# Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), # configure model
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [81]:
#To instantiate the Hyperband tuner, you must specify the hypermodel, the objective to optimize and the maximum number of epochs to train (max_epochs).
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')
# Create a callback to stop training early after reaching a certain value for the validation loss.
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
# Run the hyperparameter search. 
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 41s]
val_accuracy: 0.8856666684150696

Best val_accuracy So Far: 0.8889999985694885
Total elapsed time: 00h 14m 05s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 288 and the optimal learning rate for the optimizer
is 0.001.



In [83]:
# tim epoch tối ưu để huấn luyện mô hình với các tham số tối ưu đã tìm được ở trên
# xây dựng model với tham số tối ưu đã tìm đc
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4993 - accuracy: 0.8230 - val_loss: 0.4187 - val_accuracy: 0.8508
Epoch 2/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3762 - accuracy: 0.8633 - val_loss: 0.3672 - val_accuracy: 0.8666
Epoch 3/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3307 - accuracy: 0.8785 - val_loss: 0.3642 - val_accuracy: 0.8673
Epoch 4/50
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3067 - accuracy: 0.8869 - val_loss: 0.3537 - val_accuracy: 0.8675
Epoch 5/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2883 - accuracy: 0.8923 - val_loss: 0.3277 - val_accuracy: 0.8867
Epoch 6/50
1500/1500 [==============================] - 5s 4ms/step - loss: 0.2736 - accuracy: 0.8982 - val_loss: 0.3356 - val_accuracy: 0.8817
Epoch 7/50
1500/1500 [==============================] - 5s 4ms/step - loss: 0.2599 - accuracy: 0.9021 - val_loss: 0.3235 - val_accuracy:

In [84]:
# Khởi tạo lại siêu mô hình và train nó với epoch toi uu từ phía trên
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)


Epoch 1/48
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4993 - accuracy: 0.8232 - val_loss: 0.3946 - val_accuracy: 0.8595
Epoch 2/48
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3715 - accuracy: 0.8649 - val_loss: 0.3655 - val_accuracy: 0.8666
Epoch 3/48
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3332 - accuracy: 0.8774 - val_loss: 0.3465 - val_accuracy: 0.8763
Epoch 4/48
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3101 - accuracy: 0.8859 - val_loss: 0.3231 - val_accuracy: 0.8832
Epoch 5/48
1500/1500 [==============================] - 5s 4ms/step - loss: 0.2905 - accuracy: 0.8915 - val_loss: 0.3326 - val_accuracy: 0.8801
Epoch 6/48
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2758 - accuracy: 0.8986 - val_loss: 0.3259 - val_accuracy: 0.8819
Epoch 7/48
1500/1500 [==============================] - 5s 4ms/step - loss: 0.2611 - accuracy: 0.9023 - val_loss: 0.3282 - val_accuracy:

In [85]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 3ms/step - loss: 0.6031 - accuracy: 0.8793
[test loss, test accuracy]: [0.6030765771865845, 0.8792999982833862]
